In [43]:
import pandas as pd
from konlpy.tag import Okt
from kiwipiepy import Kiwi
import re
from emoji import core
import emoji

okt = Okt()
kiwi = Kiwi()

In [44]:
df = pd.read_csv(
    "./data/01_raw/youtube_comments_origin.csv",
    index_col=False 
)

df.head()

,video_id,comment_id,comment_text,like_count,published_at
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z


In [45]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 254637 entries, 0 to 254636
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   video_id      254637 non-null  str  
 1   comment_id    254637 non-null  str  
 2   comment_text  254516 non-null  str  
 3   like_count    254637 non-null  int64
 4   published_at  254637 non-null  str  
dtypes: int64(1), str(4)
memory usage: 9.7 MB


# 2. 데이터 전처리

## 1) datetime 열 생성

In [46]:
df["datetime"] = pd.to_datetime(df["published_at"], format="%Y-%m-%dT%H:%M:%SZ")

In [47]:
print(df["datetime"].dtypes)

datetime64[us]


In [48]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 254637 entries, 0 to 254636
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   video_id      254637 non-null  str           
 1   comment_id    254637 non-null  str           
 2   comment_text  254516 non-null  str           
 3   like_count    254637 non-null  int64         
 4   published_at  254637 non-null  str           
 5   datetime      254637 non-null  datetime64[us]
dtypes: datetime64[us](1), int64(1), str(4)
memory usage: 11.7 MB


## 2) 결측치 처리

In [49]:
df.isna().sum()

video_id          0
comment_id        0
comment_text    121
like_count        0
published_at      0
datetime          0
dtype: int64

In [50]:
null_comments = df[df['comment_text'].isnull()]
print(null_comments.head())

          video_id                  comment_id comment_text  like_count  \
7234   rDFUl2mHIW4  Ugz3BCPtRQJkdc9Q-uV4AaABAg          NaN           6   
8629   rDFUl2mHIW4  UgwErp32xCRSWSjX8AB4AaABAg          NaN           6   
9593   rDFUl2mHIW4  UgzfCHYcXvZIORu989x4AaABAg          NaN           3   
15957  rDFUl2mHIW4  UgzAyjBkzbRQlvTH2314AaABAg          NaN           0   
16982  rDFUl2mHIW4  UgzAuoSB6sE6NCgqHdx4AaABAg          NaN           0   

               published_at            datetime  
7234   2023-09-07T14:54:03Z 2023-09-07 14:54:03  
8629   2023-08-23T04:31:10Z 2023-08-23 04:31:10  
9593   2023-08-21T03:24:04Z 2023-08-21 03:24:04  
15957  2023-08-18T10:19:32Z 2023-08-18 10:19:32  
16982  2023-08-18T09:33:10Z 2023-08-18 09:33:10  


In [51]:
df_new = df.dropna().reset_index(drop=True)
df_new.isna().sum()

video_id        0
comment_id      0
comment_text    0
like_count      0
published_at    0
datetime        0
dtype: int64

In [52]:
df_new

,video_id,comment_id,comment_text,like_count,published_at,datetime
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z,2026-01-19 11:10:22
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32
...,...,...,...,...,...,...
254511,bltjk6NfDQM,Ugz6kuiT98sBtMGHOMl4AaABAg,이 영상의 키포인트: 0:23 0:55 ㅋㅋㅋ 졸귀야..,26,2021-12-09T08:12:40Z,2021-12-09 08:12:40
254512,bltjk6NfDQM,UgzToY1yV_46jAEGnft4AaABAg,Ok,2,2021-12-09T08:10:53Z,2021-12-09 08:10:53
254513,bltjk6NfDQM,Ugz8FlnMEKbhYT3EhuJ4AaABAg,Love this version 💖👍7️⃣,5,2021-12-09T08:09:07Z,2021-12-09 08:09:07
254514,bltjk6NfDQM,UgxlXbdo2QgRWbt-N_F4AaABAg,Ive are improving drastically 🤧,42,2021-12-09T08:06:42Z,2021-12-09 08:06:42


## 3) 클리닝

In [53]:
cleaned_comments = []

for comment in df_new["comment_text"]:
    if pd.isna(comment) or not isinstance(comment, str):
        cleaned_comments.append("")
        continue
    
    # 1. 이모지(그림 문자) 제거
    text = core.replace_emoji(comment,"")
    
    # 2. 한글과 공백만 남기기 (초성 'ㅋㅋㅋ', 영어, 숫자, 특수문자 모두 제거)
    text = re.sub(r'[^가-힣\s]', '', text)
    
    # 3. 불필요한 연속 공백 제거 및 양쪽 끝 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    cleaned_comments.append(text)
    

In [54]:
df_new['cleaned_comment'] = cleaned_comments

In [55]:
print(df_new[['cleaned_comment']])

                                          cleaned_comment
0       그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1                                                  눈물이 난다
2                                             이 노래도 겁나 좋네
3          패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4                                       언제 들어도 행복하고 좋은 노래
...                                                   ...
254511                                     이 영상의 키포인트 졸귀야
254512                                                   
254513                                                   
254514                                                   
254515                                                   

[254516 rows x 1 columns]


In [56]:
df_new.head()

,video_id,comment_id,comment_text,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래


In [57]:
df_new = df_new.drop(columns=["comment_text"])

In [58]:
df_new

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래
...,...,...,...,...,...,...
254511,bltjk6NfDQM,Ugz6kuiT98sBtMGHOMl4AaABAg,26,2021-12-09T08:12:40Z,2021-12-09 08:12:40,이 영상의 키포인트 졸귀야
254512,bltjk6NfDQM,UgzToY1yV_46jAEGnft4AaABAg,2,2021-12-09T08:10:53Z,2021-12-09 08:10:53,
254513,bltjk6NfDQM,Ugz8FlnMEKbhYT3EhuJ4AaABAg,5,2021-12-09T08:09:07Z,2021-12-09 08:09:07,
254514,bltjk6NfDQM,UgxlXbdo2QgRWbt-N_F4AaABAg,42,2021-12-09T08:06:42Z,2021-12-09 08:06:42,


In [59]:
df_new.to_csv(
    "./data/01_raw/processed/youtube_comments_clean.csv"
)

In [60]:
df2 = pd.read_csv(
    "./data/01_raw/processed/youtube_comments_clean.csv",
    index_col=0
)
df2.head()

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래


# 3. 데이터 탐색

In [61]:
df2["video_id"].unique()

<StringArray>
['k5O1F_IS0mU', 'rDFUl2mHIW4', 'fff8P0kYexQ', 'T_gJpnjJsnk', 'EYG4ROejmyI',
 '8L001YT4k6k', '6ZUIwj3FgUY', 'B1ShLiq3EVc', '3_b5oKGndn0', 'iUw3LPM7OBU',
 'TVqEwvbZP30', 'bltjk6NfDQM']
Length: 12, dtype: str

# 4. 토크나이징

## 1) 전체 데이터

In [173]:
df2.head()

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래


#### 명사+형용사

In [ ]:
word_list = []
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드"]

for sent in df2["cleaned_comment"]:
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = okt.pos(clean_sent)
    sub_list = []
    for res in result:
        word = res[0]
        pos = res[1]
        if word in stopwords:
            continue
        if pos in ["Noun", "Adjective"] and len(word) > 1 :
            sub_list.append(word)
    print(sub_list)
    result_str = " ".join(sub_list)
    word_list.append(sub_list)
    print("="*100)

In [174]:
stopwords = [
    "키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"
    ]

sentence_words = []
sentence_index = []

for _, row in df2.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                       # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos in ["Noun", "Adjective"] and len(word) > 1:           # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [175]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="keywords"
)

In [176]:
sentence_series.head()

k5O1F_IS0mU    [진짜, 애정, 애정, 아이돌, 처음, 콘서트, 아이돌, 처음, 진짜, 익숙하지, ...
k5O1F_IS0mU                                                 [눈물]
k5O1F_IS0mU                                             [노래, 좋네]
k5O1F_IS0mU                                    [패러블, 다시, 이쁜, 버전]
k5O1F_IS0mU                                   [언제, 행복하고, 좋은, 노래]
Name: keywords, dtype: object

In [177]:
sentence_series.to_csv("./data/01_raw/subsets/Full_youtube_comments_N_A.csv")

#### 명사

In [178]:
stopwords = [
    "키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"
    ]

sentence_words = []
sentence_index = []

for _, row in df2.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                       # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:           # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [179]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="keywords"
)

In [180]:
sentence_series.head()

k5O1F_IS0mU    [진짜, 애정, 애정, 아이돌, 처음, 콘서트, 아이돌, 처음, 진짜, 마음, 자신...
k5O1F_IS0mU                                                 [눈물]
k5O1F_IS0mU                                                 [노래]
k5O1F_IS0mU                                        [패러블, 다시, 버전]
k5O1F_IS0mU                                             [언제, 노래]
Name: keywords, dtype: object

In [181]:
sentence_series.to_csv("./data/01_raw/subsets/Full_youtube_comments_nouns.csv")

## 2) 이세계 아이돌

### (1) 최초공개무대

In [165]:
df_ISEGYE_01= df2[df2['video_id'] == 'k5O1F_IS0mU']
df_ISEGYE_01

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,0,2026-01-31T15:59:29Z,2026-01-31 15:59:29,그냥 얘들아 내가 진짜 애정한다 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트...
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,2,2026-01-23T04:29:25Z,2026-01-23 04:29:25,눈물이 난다
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,2,2026-01-20T01:52:56Z,2026-01-20 01:52:56,이 노래도 겁나 좋네
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,3,2026-01-19T11:10:22Z,2026-01-19 11:10:22,패러블 이제 뉴세돌로 리와인드 다시 찍어 올려라 이세돌 더 이쁜 버전으로 보고 싶다
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,2,2026-01-16T02:53:32Z,2026-01-16 02:53:32,언제 들어도 행복하고 좋은 노래
...,...,...,...,...,...,...
1246,k5O1F_IS0mU,Ugyx5xV_uVyS4FwcMgF4AaABAg,0,2023-12-21T17:37:00Z,2023-12-21 17:37:00,미친킹아
1247,k5O1F_IS0mU,Ugz2PXlYohe9FXhs8Md4AaABAg,0,2023-12-21T17:36:59Z,2023-12-21 17:36:59,캬
1248,k5O1F_IS0mU,UgxWD10WvFprN5Qt8pt4AaABAg,0,2023-12-21T17:36:59Z,2023-12-21 17:36:59,캬
1249,k5O1F_IS0mU,UgyuO6NYpvA-8o4CClB4AaABAg,0,2023-12-21T17:36:59Z,2023-12-21 17:36:59,NaN


In [166]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos in ["Noun", "Adjective"] and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [167]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="keywords"
)

In [168]:
sentence_series.head()

k5O1F_IS0mU    [진짜, 애정, 애정, 아이돌, 처음, 콘서트, 아이돌, 처음, 진짜, 익숙하지, ...
k5O1F_IS0mU                                                 [눈물]
k5O1F_IS0mU                                             [노래, 좋네]
k5O1F_IS0mU                                    [패러블, 다시, 이쁜, 버전]
k5O1F_IS0mU                                   [언제, 행복하고, 좋은, 노래]
Name: keywords, dtype: object

In [169]:
len(sentence_series)

1251

In [170]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_first.csv")

### (2) 최고 조회수

In [89]:
df_ISEGYE_02 = df2[df2["video_id"] == "rDFUl2mHIW4"]
df_ISEGYE_02

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
1251,rDFUl2mHIW4,Ugx28i-qwkQl3TMV7B14AaABAg,0,2026-02-02T07:18:07Z,2026-02-02 07:18:07,노래가 너무 좋네요 응원합니다
1252,rDFUl2mHIW4,UgyYR0Wqvj7MUomOcJx4AaABAg,13,2026-02-01T06:32:36Z,2026-02-01 06:32:36,년도에도 보는 사람 개추
1253,rDFUl2mHIW4,UgzHOtXKRsOuzceieYh4AaABAg,6,2026-01-31T00:35:38Z,2026-01-31 00:35:38,년에 듣는 사람
1254,rDFUl2mHIW4,Ugzs2HjOb-1kTUFApQp4AaABAg,3,2026-01-29T05:32:56Z,2026-01-29 05:32:56,NaN
1255,rDFUl2mHIW4,Ugyh3QDtNf48vl3Phbl4AaABAg,5,2026-01-29T05:30:47Z,2026-01-29 05:30:47,NaN
...,...,...,...,...,...,...
20570,rDFUl2mHIW4,UgxFJ1z5lJ95hJDnqe94AaABAg,0,2023-08-18T04:30:15Z,2023-08-18 04:30:15,큰거오나
20571,rDFUl2mHIW4,Ugxhv_RuSpzasuggjpR4AaABAg,0,2023-08-18T04:30:15Z,2023-08-18 04:30:15,너무좋다
20572,rDFUl2mHIW4,UgwAg59HI_vwZcqDBup4AaABAg,0,2023-08-18T04:30:13Z,2023-08-18 04:30:13,킹아
20573,rDFUl2mHIW4,UgywI8hniF6umfe-YWp4AaABAg,0,2023-08-18T04:30:02Z,2023-08-18 04:30:02,오우


In [90]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [91]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [92]:
sentence_series.head()

rDFUl2mHIW4        [노래, 응원]
rDFUl2mHIW4    [년도, 사람, 개추]
rDFUl2mHIW4            [사람]
rDFUl2mHIW4            None
rDFUl2mHIW4            None
Name: nouns, dtype: object

In [93]:
len(sentence_series)

19324

In [94]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_most.csv")

### (3) 최신 컴백

In [95]:
df_ISEGYE_03 = df2[df2['video_id'] == 'fff8P0kYexQ']
df_ISEGYE_03 

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
20575,fff8P0kYexQ,UgwdWrDpJo2BWg28AWt4AaABAg,4,2026-02-02T02:16:39Z,2026-02-02 02:16:39,나 탈덕한 줄 알았는데 아니었네
20576,fff8P0kYexQ,UgwesPoV9K_9mbPPEEF4AaABAg,1,2026-02-01T16:00:52Z,2026-02-01 16:00:52,근데 베이스 지리긴한다
20577,fff8P0kYexQ,Ugykhnkil5dRTerVKvd4AaABAg,11,2026-01-31T02:38:16Z,2026-01-31 02:38:16,진짜 내 인생곡 이세돌이여 영원하라
20578,fff8P0kYexQ,UgzPe-6nC6oAQNLh-JV4AaABAg,8,2026-01-30T17:28:31Z,2026-01-30 17:28:31,아위할때 개이쁘당
20579,fff8P0kYexQ,UgzwwL3vmd_h0zDHG0Z4AaABAg,12,2026-01-30T14:39:48Z,2026-01-30 14:39:48,그냥 노래가 좋다 엄청 잘어울리네
...,...,...,...,...,...,...
24408,fff8P0kYexQ,UgwEjRH_BRGD1jT5L6d4AaABAg,0,2025-11-06T09:00:28Z,2025-11-06 09:00:28,드디어
24409,fff8P0kYexQ,Ugzro5UtqNodrpym75V4AaABAg,0,2025-11-06T09:00:27Z,2025-11-06 09:00:27,올라왔구나 드디어
24410,fff8P0kYexQ,Ugwq0jvjea2qY6NhSz94AaABAg,0,2025-11-06T09:00:27Z,2025-11-06 09:00:27,떴다
24411,fff8P0kYexQ,Ugx4iqRswJSVHT4TBjp4AaABAg,2,2025-11-06T09:00:24Z,2025-11-06 09:00:24,NaN


In [96]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_ISEGYE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [97]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [98]:
sentence_series.head()

fff8P0kYexQ        [탈덕]
fff8P0kYexQ       [베이스]
fff8P0kYexQ    [진짜, 인생]
fff8P0kYexQ        [아위]
fff8P0kYexQ        [노래]
Name: nouns, dtype: object

In [99]:
len(sentence_series)

3838

In [100]:
sentence_series.to_csv("./data/01_raw/subsets/ISEGYE_comments_recent.csv")

## 3. 플레이브

### 1) 최초공개무대

In [101]:
df_PLAVE_01 = df2[df2['video_id'] == 'T_gJpnjJsnk']
df_PLAVE_01

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
24413,T_gJpnjJsnk,Ugwmr8Ar6EQGs5xhwYh4AaABAg,0,2026-02-01T20:47:21Z,2026-02-01 20:47:21,팬 아닌사람들은 영원히 모르겠지만 저는 플레이브가 각자의 서사를 숨기지 않고 이렇게...
24414,T_gJpnjJsnk,Ugz8V7mzu8AAVYQWhgt4AaABAg,0,2026-01-31T23:59:56Z,2026-01-31 23:59:56,NaN
24415,T_gJpnjJsnk,Ugyk-xIfpb4g-KTfgLl4AaABAg,2,2026-01-31T19:32:15Z,2026-01-31 19:32:15,내 꿈도 저렇게 빛났으면 좋겠다
24416,T_gJpnjJsnk,Ugw2PR0vzU5HNyxxLR54AaABAg,1,2026-01-27T12:58:09Z,2026-01-27 12:58:09,오랜만에 돌아왔다아국가어떻게 데뷔곡이 기다릴게 어떻게 데뷔곡이 자체제작 심지어 안무...
24417,T_gJpnjJsnk,UgyRzeLz69WVLFQDBcZ4AaABAg,0,2026-01-24T13:11:14Z,2026-01-24 13:11:14,기다릴게 너의 그곳에 내가 닿을 수 있게가 데뷔곡 가사라니 그때 멤버들의 간절한 마...
...,...,...,...,...,...,...
30426,T_gJpnjJsnk,UgxuNJWBpz8ESubkIRV4AaABAg,4,2023-03-18T07:11:41Z,2023-03-18 07:11:41,NaN
30427,T_gJpnjJsnk,UgwR6k_rzeajbbuVvMF4AaABAg,5,2023-03-18T07:09:54Z,2023-03-18 07:09:54,NaN
30428,T_gJpnjJsnk,UgyZLFkm-zYoikl5o8x4AaABAg,10,2023-03-18T07:01:23Z,2023-03-18 07:01:23,얘들아 무대잘봤고 수고해 기특해라
30429,T_gJpnjJsnk,UgywxQjut3ANePQRSpd4AaABAg,6,2023-03-18T06:59:07Z,2023-03-18 06:59:07,NaN


In [102]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [103]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [104]:
sentence_series.head()

T_gJpnjJsnk    [사람, 레이브, 각자, 서사, 숨기, 도움, 생각, 버추, 하나, 타격, 영영, ...
T_gJpnjJsnk                                                 None
T_gJpnjJsnk                                                 None
T_gJpnjJsnk            [국가, 데뷔, 데뷔, 자체, 제작, 심지어, 안무, 요지, 플하, 사랑]
T_gJpnjJsnk                            [데뷔, 가사, 그때, 멤버, 마음, 갑자기]
Name: nouns, dtype: object

In [105]:
len(sentence_series)

6018

In [106]:
sentence_series.to_csv("./data/01_raw/subsets/PLAVE_comments_first.csv")

### 2) 최고 조회수

In [182]:
df_PLAVE_02 = df2[df2['video_id'] == 'EYG4ROejmyI']
df_PLAVE_02

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
30431,EYG4ROejmyI,UgzVM6ti1F291mPYdvZ4AaABAg,19493,2024-08-20T13:18:06Z,2024-08-20 13:18:06,안녕하세요 얼굴천재 차은수입니다 플레이브와 함께 해서 영광이에요 펌업더볼륨 위 할 ...
30432,EYG4ROejmyI,Ugw0DsglBQ_uBtlt3M14AaABAg,0,2026-02-02T07:17:55Z,2026-02-02 07:17:55,NaN
30433,EYG4ROejmyI,UgwQtIcAFWdcsTN-PSF4AaABAg,33,2026-01-31T01:48:10Z,2026-01-31 01:48:10,NaN
30434,EYG4ROejmyI,UgwKB5WTzcW5_78I_2h4AaABAg,0,2026-01-29T07:12:43Z,2026-01-29 07:12:43,푸티비 만들던 라이브 생각날 때마다 보는데 봐도봐도 기분이 짜릿함
30435,EYG4ROejmyI,UgwDF1eVkeLyDkn7tKh4AaABAg,3,2026-01-26T15:19:40Z,2026-01-26 15:19:40,솔직히 처음 플레이브 봤을때는 거부감 들었는데 노래 듣다 보니까 자연스레 응원하게 된다
...,...,...,...,...,...,...
48475,EYG4ROejmyI,UgwloeBa4UgZ3BXCkqt4AaABAg,1,2024-08-20T08:30:50Z,2024-08-20 08:30:50,NaN
48476,EYG4ROejmyI,Ugyn7a623mn6viqfbll4AaABAg,1,2024-08-20T08:30:45Z,2024-08-20 08:30:45,왐마
48477,EYG4ROejmyI,UgzR8USxVPIorXLWRI54AaABAg,21,2024-08-20T08:30:42Z,2024-08-20 08:30:42,와우
48478,EYG4ROejmyI,Ugy-CYDwPwISu-4L0gB4AaABAg,1,2024-08-20T08:30:42Z,2024-08-20 08:30:42,NaN


In [183]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [184]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [185]:
sentence_series.head()

EYG4ROejmyI    [얼굴, 천재, 은수, 레이브, 영광, 볼륨]
EYG4ROejmyI                         None
EYG4ROejmyI                         None
EYG4ROejmyI                [티비, 라이브, 기분]
EYG4ROejmyI        [처음, 레이브, 거부, 노래, 응원]
Name: nouns, dtype: object

In [111]:
len(sentence_series)

18049

In [186]:
sentence_series.to_csv("./data/01_raw/subsets/PLAVE_comments_most.csv")

### 3) 최신 컴백

In [113]:
df_PLAVE_03 = df2[df2['video_id'] == '8L001YT4k6k']
df_PLAVE_03

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
48480,8L001YT4k6k,Ugwc9ZGQ_RElvVSl9ih4AaABAg,0,2026-02-01T15:13:13Z,2026-02-01 15:13:13,NaN
48481,8L001YT4k6k,Ugw77q0MPeoPH3P2J-R4AaABAg,2,2026-02-01T01:49:00Z,2026-02-01 01:49:00,진심 멜로디가 사기
48482,8L001YT4k6k,UgyRrLSmfW70x18zWI94AaABAg,1,2026-01-31T13:56:11Z,2026-01-31 13:56:11,헐랭 님들 키티랑 하미니 생일 똑같아요
48483,8L001YT4k6k,UgzBSHev1NttzeOsun14AaABAg,0,2026-01-31T05:55:28Z,2026-01-31 05:55:28,NaN
48484,8L001YT4k6k,UgzjQ9AUxUOGucpjl2V4AaABAg,5,2026-01-29T13:54:47Z,2026-01-29 13:54:47,NaN
...,...,...,...,...,...,...
66927,8L001YT4k6k,Ugx34V3BiB-CSnGb7Mh4AaABAg,0,2025-11-10T09:00:24Z,2025-11-10 09:00:24,NaN
66928,8L001YT4k6k,UgykGUT8V_0BttbSuaV4AaABAg,0,2025-11-10T09:00:23Z,2025-11-10 09:00:23,NaN
66929,8L001YT4k6k,UgxZFCVmllWVjaph3XN4AaABAg,9,2025-11-10T09:00:22Z,2025-11-10 09:00:22,플레이브 귀엽다라는 생각이 멈추지 않는 탓일까
66930,8L001YT4k6k,Ugwylin0tbXNYudygy14AaABAg,125,2025-11-10T09:00:21Z,2025-11-10 09:00:21,NaN


In [114]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_PLAVE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [115]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [116]:
sentence_series.head()

8L001YT4k6k                None
8L001YT4k6k       [진심, 멜로디, 사기]
8L001YT4k6k    [헐랭, 키티, 하미, 생일]
8L001YT4k6k                None
8L001YT4k6k                None
Name: nouns, dtype: object

In [117]:
len(sentence_series)

18452

In [118]:
sentence_series.to_csv("./data/01_raw/subsets/PLAVE_comments_recent.csv")

## 4. 아이브

### 1) 최초 공개 무대

In [121]:
df_IVE_01 = df2[df2['video_id'] == 'bltjk6NfDQM']
df_IVE_01

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
253004,bltjk6NfDQM,Ugw-HcESiZIXJI_2n0d4AaABAg,0,2026-02-03T01:18:15Z,2026-02-03 01:18:15,NaN
253005,bltjk6NfDQM,Ugw1SlNZWN9Z8DxRGeV4AaABAg,0,2026-02-01T02:41:28Z,2026-02-01 02:41:28,저거다 기계라서 기계 한테 보나스 줘야함
253006,bltjk6NfDQM,UgxkGWBOmpTcknO0dnd4AaABAg,0,2026-01-28T17:10:22Z,2026-01-28 17:10:22,NaN
253007,bltjk6NfDQM,UgzrPBnqF7GgEkofxzV4AaABAg,9,2026-01-27T02:34:30Z,2026-01-27 02:34:30,혹시 에 있는 사람
253008,bltjk6NfDQM,Ugw_uSrn9L9LSPizdyx4AaABAg,0,2026-01-23T18:17:27Z,2026-01-23 18:17:27,NaN
...,...,...,...,...,...,...
254511,bltjk6NfDQM,Ugz6kuiT98sBtMGHOMl4AaABAg,26,2021-12-09T08:12:40Z,2021-12-09 08:12:40,이 영상의 키포인트 졸귀야
254512,bltjk6NfDQM,UgzToY1yV_46jAEGnft4AaABAg,2,2021-12-09T08:10:53Z,2021-12-09 08:10:53,NaN
254513,bltjk6NfDQM,Ugz8FlnMEKbhYT3EhuJ4AaABAg,5,2021-12-09T08:09:07Z,2021-12-09 08:09:07,NaN
254514,bltjk6NfDQM,UgxlXbdo2QgRWbt-N_F4AaABAg,42,2021-12-09T08:06:42Z,2021-12-09 08:06:42,NaN


In [122]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_IVE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [123]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [124]:
sentence_series.head()

bltjk6NfDQM            None
bltjk6NfDQM    [기계, 기계, 나스]
bltjk6NfDQM            None
bltjk6NfDQM        [혹시, 사람]
bltjk6NfDQM            None
Name: nouns, dtype: object

In [125]:
len(sentence_series)

1512

In [126]:
sentence_series.to_csv("./data/01_raw/subsets/IVE_comments_first.csv")

### 2) 최고 조회수

In [127]:
df_IVE_02 = df2[df2['video_id'] == '6ZUIwj3FgUY']
df_IVE_02

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
66932,6ZUIwj3FgUY,UgxJvLxGUKrWPAETuL94AaABAg,0,2026-02-02T07:58:11Z,2026-02-02 07:58:11,아이브 노래좋아요
66933,6ZUIwj3FgUY,UgwK5_3kMQKcLyO2i6Z4AaABAg,0,2026-02-02T07:10:11Z,2026-02-02 07:10:11,NaN
66934,6ZUIwj3FgUY,UgxgNb5120j1fXHHLK94AaABAg,7,2026-02-02T01:03:46Z,2026-02-02 01:03:46,NaN
66935,6ZUIwj3FgUY,UgzyYrD3CM7JKcXgM3F4AaABAg,1,2026-02-02T00:22:14Z,2026-02-02 00:22:14,분에 도착합니다 오늘
66936,6ZUIwj3FgUY,UgwwR8BtuuCO6oDNu_x4AaABAg,0,2026-02-01T21:03:25Z,2026-02-01 21:03:25,NaN
...,...,...,...,...,...,...
174923,6ZUIwj3FgUY,Ugz1kiW89md5qsLAJB94AaABAg,0,2023-04-10T09:00:18Z,2023-04-10 09:00:18,NaN
174924,6ZUIwj3FgUY,UgzQ6RnSnxhxR6IZ0YR4AaABAg,0,2023-04-10T09:00:18Z,2023-04-10 09:00:18,다들 음색 미쳣따 우리 짱이브 요번 활동도 화이팅 안전하게 잘 끝내쟈 사랑해
174925,6ZUIwj3FgUY,Ugw1v6yo4q4cddnE_iZ4AaABAg,1,2023-04-10T09:00:18Z,2023-04-10 09:00:18,오오오오옹 아이브 계속 기다렸당
174926,6ZUIwj3FgUY,UgyanOZRVaE7R0MjA0V4AaABAg,1,2023-04-10T09:00:18Z,2023-04-10 09:00:18,갓기들 진짜 사랑해 너무 예뻐


In [128]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_IVE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [129]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [130]:
sentence_series.head()

bltjk6NfDQM            None
bltjk6NfDQM    [기계, 기계, 나스]
bltjk6NfDQM            None
bltjk6NfDQM        [혹시, 사람]
bltjk6NfDQM            None
Name: nouns, dtype: object

In [131]:
len(sentence_series)

1512

In [132]:
sentence_series.to_csv("./data/01_raw/subsets/IVE_comments_most.csv")

### 2) 최신 컴백

In [133]:
df_IVE_03 = df2[df2['video_id'] == 'B1ShLiq3EVc']
df_IVE_03

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
174928,B1ShLiq3EVc,UgxVCkjnchBj0gkR1CV4AaABAg,0,2026-02-02T02:10:11Z,2026-02-02 02:10:11,잘못하면 유치해질수 있는 컨셉인데유치하지않고 키치하면서 판타지스럽게 잘 연출한듯뮤비...
174929,B1ShLiq3EVc,UgxBj1itLGZi_vbQVP94AaABAg,0,2026-02-01T18:16:10Z,2026-02-01 18:16:10,NaN
174930,B1ShLiq3EVc,UgzmAnrbpJIuDU-2L2R4AaABAg,2,2026-02-01T18:15:20Z,2026-02-01 18:15:20,NaN
174931,B1ShLiq3EVc,UgzDAZEfRi_3M5aFZSp4AaABAg,0,2026-02-01T16:01:29Z,2026-02-01 16:01:29,NaN
174932,B1ShLiq3EVc,Ugyk5OvVKjF9WhK-_FR4AaABAg,6,2026-02-01T14:39:47Z,2026-02-01 14:39:47,NaN
...,...,...,...,...,...,...
200274,B1ShLiq3EVc,UgwxVn6x7m_bZAPgui14AaABAg,0,2025-08-25T09:00:17Z,2025-08-25 09:00:17,꺄악
200275,B1ShLiq3EVc,Ugxlcpt0rMnYK5pFaOh4AaABAg,0,2025-08-25T09:00:17Z,2025-08-25 09:00:17,일찍 옴
200276,B1ShLiq3EVc,UgzD9nJre_c9jaSu9bx4AaABAg,2,2025-08-25T09:00:17Z,2025-08-25 09:00:17,빠 썸네일에 원영이 머리 넘 귀엽
200277,B1ShLiq3EVc,UgxkVN4GqZQsRnlz0KN4AaABAg,1,2025-08-25T09:00:16Z,2025-08-25 09:00:16,NaN


In [134]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_IVE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [135]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [136]:
sentence_series.head()

6ZUIwj3FgUY    [아이브, 노래]
6ZUIwj3FgUY         None
6ZUIwj3FgUY         None
6ZUIwj3FgUY     [도착, 오늘]
6ZUIwj3FgUY         None
Name: nouns, dtype: object

In [137]:
len(sentence_series)

107996

In [138]:
sentence_series.to_csv("./data/01_raw/subsets/IVE_comments_recent.csv")

## 5. 라이즈

### 1) 최초공개무대

In [139]:
df_RIIZE_01 = df2[df2['video_id'] == '3_b5oKGndn0']
df_RIIZE_01

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
200279,3_b5oKGndn0,Ugyj6wMbDC740iiWtt94AaABAg,2,2026-01-30T06:06:53Z,2026-01-30 06:06:53,에도 오는 브리즈 이 느낌 잃지말아줘 제발
200280,3_b5oKGndn0,UgzoDzdkzpBmPVFHzGx4AaABAg,0,2026-01-15T00:04:55Z,2026-01-15 00:04:55,NaN
200281,3_b5oKGndn0,Ugx47QlArGl5Tw1NyCF4AaABAg,0,2026-01-13T17:50:12Z,2026-01-13 17:50:12,이때 컨셉으로 돌아와
200282,3_b5oKGndn0,UgzujHkRhnlG9So-yZt4AaABAg,0,2026-01-04T19:02:04Z,2026-01-04 19:02:04,NaN
200283,3_b5oKGndn0,UgzP_xs_1AszU4t59iF4AaABAg,0,2026-01-03T23:40:24Z,2026-01-03 23:40:24,년에도
...,...,...,...,...,...,...
201539,3_b5oKGndn0,UgxzpekCRPwbHvNe_ol4AaABAg,10,2023-09-07T10:37:29Z,2023-09-07 10:37:29,NaN
201540,3_b5oKGndn0,UgwbQXROYsx7hue3T3F4AaABAg,142,2023-09-07T10:37:21Z,2023-09-07 10:37:21,NaN
201541,3_b5oKGndn0,Ugx2LaA59wCVhokZiD54AaABAg,0,2023-09-07T10:36:58Z,2023-09-07 10:36:58,NaN
201542,3_b5oKGndn0,Ugx65pSKBagRtAWpVhp4AaABAg,0,2023-09-07T10:36:45Z,2023-09-07 10:36:45,NaN


In [140]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_01.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [141]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [142]:
sentence_series.head()

3_b5oKGndn0    [브리, 느낌]
3_b5oKGndn0        None
3_b5oKGndn0        [컨셉]
3_b5oKGndn0        None
3_b5oKGndn0        None
Name: nouns, dtype: object

In [143]:
len(sentence_series)

1265

In [144]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_first.csv")

### 2) 최고 조회수

In [145]:
df_RIIZE_02 = df2[df2['video_id'] == 'iUw3LPM7OBU']
df_RIIZE_02

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
201544,iUw3LPM7OBU,UgzpysFyiOuu9vSDiMd4AaABAg,1,2026-02-01T10:34:39Z,2026-02-01 10:34:39,NaN
201545,iUw3LPM7OBU,Ugwzwlsx2nMXe1KoyDN4AaABAg,0,2026-01-31T13:31:54Z,2026-01-31 13:31:54,NaN
201546,iUw3LPM7OBU,UgwKLUPhFAkUZ5WzC2p4AaABAg,0,2026-01-31T13:31:00Z,2026-01-31 13:31:00,NaN
201547,iUw3LPM7OBU,UgzXEsifbaHcEIID72l4AaABAg,0,2026-01-31T13:29:21Z,2026-01-31 13:29:21,NaN
201548,iUw3LPM7OBU,UgzIxvdDdWjzMsoNtQR4AaABAg,0,2026-01-31T13:29:06Z,2026-01-31 13:29:06,NaN
...,...,...,...,...,...,...
238034,iUw3LPM7OBU,UgzC1820HI7R_66CN0l4AaABAg,0,2023-09-04T09:00:43Z,2023-09-04 09:00:43,NaN
238035,iUw3LPM7OBU,UgyzJMB4ItFCDo2QzC54AaABAg,1,2023-09-04T09:00:40Z,2023-09-04 09:00:40,NaN
238036,iUw3LPM7OBU,UgzA14ijlRm7rGzX6bV4AaABAg,0,2023-09-04T09:00:38Z,2023-09-04 09:00:38,NaN
238037,iUw3LPM7OBU,UgyyUzmtfUi-kZELVVp4AaABAg,147,2023-09-04T09:00:35Z,2023-09-04 09:00:35,NaN


In [146]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_02.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [147]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [148]:
sentence_series.head()

iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
iUw3LPM7OBU    None
Name: nouns, dtype: object

In [149]:
len(sentence_series)

36495

In [150]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_most.csv")

### 3) 최신 컴백

In [151]:
df_RIIZE_03 = df2[df2['video_id'] == 'TVqEwvbZP30']
df_RIIZE_03

,video_id,comment_id,like_count,published_at,datetime,cleaned_comment
238039,TVqEwvbZP30,UgwQgzuHZXtzXe-gBod4AaABAg,0,2026-02-02T04:26:02Z,2026-02-02 04:26:02,NaN
238040,TVqEwvbZP30,UgyH02NPr3Cej19pjy14AaABAg,0,2026-02-02T04:20:47Z,2026-02-02 04:20:47,NaN
238041,TVqEwvbZP30,Ugyi_vZq7KHEEmwJ7t54AaABAg,2,2026-02-02T02:01:36Z,2026-02-02 02:01:36,NaN
238042,TVqEwvbZP30,UgxOZ8SqQBN94w7Z6d54AaABAg,3,2026-02-02T01:23:41Z,2026-02-02 01:23:41,이거 레알 커리어하이급인데 왜 다들 몰라주는거지 진심 라이즈 노래중에 탑쓰리로 좋은데
238043,TVqEwvbZP30,UgzQS6lO44-awd2oJqF4AaABAg,3,2026-02-01T21:52:39Z,2026-02-01 21:52:39,NaN
...,...,...,...,...,...,...
252999,TVqEwvbZP30,Ugwm7jVlfsoBlyx2isB4AaABAg,1,2025-11-24T09:00:25Z,2025-11-24 09:00:25,NaN
253000,TVqEwvbZP30,Ugyz3kSw070Uz9sU0Ld4AaABAg,188,2025-11-24T09:00:25Z,2025-11-24 09:00:25,NaN
253001,TVqEwvbZP30,UgzGeXjlTJAOCqIUNtB4AaABAg,1,2025-11-24T09:00:25Z,2025-11-24 09:00:25,NaN
253002,TVqEwvbZP30,Ugy81xNDbdlJVTVRadx4AaABAg,1,2025-11-24T09:00:22Z,2025-11-24 09:00:22,NaN


In [152]:
stopwords = ["키키", "샤갈", "이세돌", "와인", "리와인드", "들보", "이후", "싯팔", "흑흐긓극",
    "나노", "이건", "땃다", "꾸엪", "존나", "띠링띠링띠링", "아아", "개재", "론데",
    "년반", "쭉쭉", "하쥰내", "시진핑", "아앙", "런애내댤멷", "흐규흐규", "어유", "삉삉",
    "쁴비븨빙", "킹애", "하니", "그것", "그거", "새끼", "젠장", "건가", "뭐임", "가기", "목따겄",
    "다왜우", "오오오오오오오", "흐흐", "이세", "가끔", "가즈", "이분", "무럭무럭", "꿀맛이쟈", "이면",
    "보고", "뭐노", "구구국", "구국", "구구", "구구구구구구구", "그냥", "존나", "쥰내", "게뭐", "도티잠뜰",
    "노무현", "이기", "구독", "래야", "걍쩌는노", "븅신", "비챤츕츕", "예스", "에아", "스널", "시발", "구나",
    "이노", "오오오", "쭉쭉", "오오오오", "레잔", "로써", "모두", "레게", "무니", "정도", "정말", "어쨋", "마냥",
    "유튜므", "계속", "가야", "부터", "당신", "수가", "만가", "정도", "더더", "이보", "시오", "거구", "큿소",
    "어의", "아이", "씌발", "이노", "캣맘충", "이내", "졸앟", "난날", "거제", "패토", "시발", "시바", "퀴륑",
    "뀌링", "뭐임", "미리", "이지", "이제", "이번", "제발", "이건", "게뭐", "면서", "만이", "내겐", "어쩌면", 
    "이이", "다해", "그냥", "어우", "훤해", "님들", "하디", "왜케", "부로", "조금", "승하", "쇼쇼", "어후",
    "먼처출케", "칵해퐈", "무현", "게뭐노"]

sentence_words = []
sentence_index = []

for _, row in df_RIIZE_03.iterrows():                           # 1. 데이터프레임의 행을 하나씩 반복하며 꺼냄
    sent = row["cleaned_comment"]                          # 2. 분석할 댓글 원문 추출
    comment_id = row["video_id"]                        # 3. 댓글과 연결된 비디오 ID(인덱스) 저장

    if pd.isna(sent):
        sentence_words.append(None)                     # 4. 빈 댓글(NaN)인 경우 처리
        sentence_index.append(comment_id)               # 5. 결과 리스트에 빈 값 추가 후 다음으로 넘어감
        continue

    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)   # 6. 특수문자/숫자 제거(한글, 영어, 공백만 남김)
    result = okt.pos(clean_sent)                        # 7. 형태소 분석기를 이용해 품사 태깅 수행

    nouns = []
    for word, pos in result:
        if pos == "Noun" and len(word) > 1:             # 8. 명사(Noun)이면서 길이가 2자 이상이고
            if word in stopwords:                       # 9. 미리 정의한 불용어(stopwords)에
                continue
            nouns.append(word)                          # 10. 포함되지 않는 단어만 리스트로 생성

    # 👉 핵심: 한 문장 단위
    sentence_words.append(nouns if nouns else None)
    sentence_index.append(comment_id)

In [153]:
sentence_series = pd.Series(
    sentence_words,
    index=sentence_index,
    name="nouns"
)

In [154]:
sentence_series.head()

TVqEwvbZP30                          None
TVqEwvbZP30                          None
TVqEwvbZP30                          None
TVqEwvbZP30    [레알, 커리어, 하이, 진심, 라이즈, 노래]
TVqEwvbZP30                          None
Name: nouns, dtype: object

In [155]:
sentence_series.to_csv("./data/01_raw/subsets/RIIZE_comments_recent.csv")